In [1]:
from tagger.models.received_item import ReceivedItem, ReceivedItemStatus
from tagger.core.mongo.models.order import OrderCodeMap, Order, OrderStatus
from tagger.viewsets.order.change_status import ChangeStatusSerializer, _change_status
from django.contrib.auth.models import User
from tagger.models.inventory import Inventory, InventoryStatus

import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
import pandas as pd

def split_codes(raw_codes):
    codes = []
    cursor = 0
    while cursor < len(raw_codes):
        codes.append(raw_codes[cursor: cursor + 5])
        cursor = cursor + 5
    return codes

In [3]:
with open("../Downloads/확장주문검색_20211109_151907.xlsx", "rb") as f:
    df = pd.read_excel(f, dtype=str)
    df.dropna(subset=["주문번호"], inplace=True)
    for _, row in df.iterrows():
        print(split_codes(row["주문번호"]), row["판매처 상품명"], row["수령자이름"])

['U6PUD'] 오버사이즈 와이드 칼라 코트(라임) 주보배
['CGQ4H'] Torto sweatshirt(네이비) 윤인주
['A6J8J'] 벨티드 플랩 코트(카멜) 하준희
['CUCEG'] 폭스 퍼 후드 벨티드 롱 구스 다운 정민지
['KWVD6'] 덤블 블록 페이크 무스탕 점퍼(베이지) 윤경
['VL4WR'] 폭스 퍼 후드 벨티드 롱 구스 다운 신지혜
['TKVHG', 'DVAQ6'] 핸드메이드 히든 버튼 코트, 리브 퀄팅 구스 다운 점퍼(블랙) 김미진
['K7ERU'] 더블 핸드메이드 자켓 김주연
['7TD77'] 하이넥 리버시블 무스탕 코트 홍인경
['NMVHJ', 'BQ3NU'] 리브 퀄팅 구스 다운 점퍼(블랙) , 투웨이 점퍼&조끼 다운 베스트(블랙) 민은정
['BPGMX'] [빈폴스포츠]푸퍼 미드 다운 점퍼(네이비) 채은희
['JTNUG', 'UUAGX'] [빈폴스포츠] 투웨이 점퍼&조끼 다운 베스트(화이트) , [빈폴스포츠] 투웨이 점퍼&조끼 다운 베스트(블랙) 이정원
['YRWJB'] 퀄팅 구스다운 점퍼(BLACK) 정나영
['UEWKE', '8RDAR'] 모던 벨티드 트렌치 코트_TWWBYK70030외 1 건, 와이드 칼라 앙고라 울 코트 (CAMEL) 김민선


In [5]:
for _, row in df.iterrows():
    codes = split_codes(row["주문번호"])
    tracking_no = f"{row['택배사']} {row['송장번호']}"
    tracking_url = f"https://search.naver.com/search.naver?query={tracking_no}"
    for code in codes:
        code_map = OrderCodeMap.objects(code=f"ORD-{code}").first()
        orderid = str(code_map.orderid)
        order = Order.objects.get(id=orderid)
        s = ChangeStatusSerializer(data={
            "status": OrderStatus.DELIVERY_STARTED,
            "delivery_tracking_number": tracking_no,
            "delivery_tracking_url": tracking_url,
        }, instance=order)
        s.is_valid(raise_exception=True)
        _change_status(s, order, User.objects.get(username="backoffice_server"), id=orderid)
        print(f"https://office.alloff.co/orders/{orderid}", order.payment.buyername)

https://office.alloff.co/orders/617a795b17fc2e5a1f9eab42 주보배
https://office.alloff.co/orders/617fcaf517fc2e5a1f9eb2e6 윤인주
https://office.alloff.co/orders/6180d7e817fc2e5a1f9eb4ea 하준희
https://office.alloff.co/orders/6183a76a26685ff051755184 정민지
https://office.alloff.co/orders/6183a95226685ff05175518d 윤경
https://office.alloff.co/orders/6183b82426685ff0517551bc 신지혜
https://office.alloff.co/orders/6184d8b326685ff05175540a 김미진
https://office.alloff.co/orders/6185165726685ff0517554c2 김미진
https://office.alloff.co/orders/6184ef7e26685ff05175542f 김주연
https://office.alloff.co/orders/6185018c26685ff051755440 홍인경 
https://office.alloff.co/orders/618516ef26685ff0517554c5 민은정
https://office.alloff.co/orders/61851cfd26685ff0517554eb 민은정
https://office.alloff.co/orders/6185323426685ff051755544 채은희
https://office.alloff.co/orders/61853ac026685ff051755565 이정원
https://office.alloff.co/orders/61853b3026685ff051755569 이정원
https://office.alloff.co/orders/6185ace326685ff051755600 정나영
https://office.alloff.co

In [6]:
for _, row in df.iterrows():
    codes = split_codes(row["주문번호"])
    for code in codes:
        order = Order.get_by_code(code)
        if order is not None:
            inventories = Inventory.objects.filter(status=InventoryStatus.SHIPPING_PENDING, out_order_id=str(order.id)).all()
            for i in inventories:
                print(f"""{order.code} ({order.id}) - {i.product_name} has been shipped!""")
                i.status = InventoryStatus.SHIPPED
                i.save()

ORD-U6PUD (617a795b17fc2e5a1f9eab42) - 오버사이즈 와이드 칼라 코트(라임) has been shipped!
ORD-CGQ4H (617fcaf517fc2e5a1f9eb2e6) - Torto sweatshirt(네이비) has been shipped!
ORD-A6J8J (6180d7e817fc2e5a1f9eb4ea) - 벨티드 플랩 코트(카멜)  has been shipped!
ORD-CUCEG (6183a76a26685ff051755184) - 폭스 퍼 후드 벨티드 롱 구스 다운  has been shipped!
ORD-KWVD6 (6183a95226685ff05175518d) - 덤블 블록 페이크 무스탕 점퍼(베이지)  has been shipped!
ORD-VL4WR (6183b82426685ff0517551bc) - 폭스 퍼 후드 벨티드 롱 구스 다운 has been shipped!
ORD-TKVHG (6184d8b326685ff05175540a) - 핸드메이드 히든 버튼 코트 has been shipped!
ORD-DVAQ6 (6185165726685ff0517554c2) - 리브 퀄팅 구스 다운 점퍼(블랙)  has been shipped!
ORD-K7ERU (6184ef7e26685ff05175542f) - 더블 핸드메이드 자켓  has been shipped!
ORD-7TD77 (6185018c26685ff051755440) - 하이넥 리버시블 무스탕 코트 has been shipped!
ORD-NMVHJ (618516ef26685ff0517554c5) - 리브 퀄팅 구스 다운 점퍼(블랙)  has been shipped!
ORD-BQ3NU (61851cfd26685ff0517554eb) - 투웨이 점퍼&조끼 다운 베스트(블랙)  has been shipped!
ORD-BPGMX (6185323426685ff051755544) - [빈폴스포츠]푸퍼 미드 다운 점퍼(네이비)  has been shipped!
ORD-JTN

True